# Concise Example Capacity Test using pvcaptest

This example performs the same test as the Complete Capacity Testing example, but uses the `run_test` function to apply the filters and then displays the filtering steps visually using the `scatter_filters` method.

## Imports

In [ ]:
import pandas as pd

# import captest as pvc
from captest import capdata as pvc
from bokeh.io import output_notebook, show

# uncomment below two lines to use cptest.scatter_hv in notebook
import holoviews as hv
hv.extension('bokeh')

#if working offline with the CapData.plot() method may fail
#run 'export BOKEH_RESOURCES=inline' at the command line before
#running the jupyter notebook

output_notebook()

## Load and Plot Measured Data

We begin by instantiating a `CapData` object, which we will use to load and store the measured data.  In this example we will calculate reporting conditions from the measured data, so we load and filter the measured data first.

In [ ]:
das = pvc.CapData('das')

The `load_data` method by default will look for and attempt to load all files ending with '.csv' in a 'data' folder.  In this case we have a single file and provide the filename, so only the file specified is loaded.

In [ ]:
das.load_data(fname='example_meas_data.csv', source='AlsoEnergy')

In [ ]:
das.set_regression_cols(power='-mtr-', poa='irr-poa-', t_amb='temp-amb-', w_vel='wind--')

In [ ]:
das.agg_sensors(agg_map={'-inv-':'sum', 'irr-poa-':'mean', 'temp-amb-':'mean', 'wind--':'mean'},
                inv_sum_vs_power=False)

In [ ]:
das.plot(marker='line', width=900, height=250, ncols=1)

## Filtering Measured Data
The `CapData` class provides a number of convience methods to apply filtering steps as defined in ASTM E2848.  The following section demonstrates the use of the more commonly used filtering steps to remove measured data points.

In [ ]:
# Uncomment and run to copy over the filtered dataset with the unfiltered data.
das.reset_filter()

In [ ]:
measured_filters = [
    (pvc.CapData.filter_sensors, (), {}),
    (pvc.CapData.filter_custom, (pd.DataFrame.dropna, ), {}),
    (pvc.CapData.filter_irr, (200, 2000), {}),
    (pvc.CapData.filter_outliers, (), {}),
    (pvc.CapData.fit_regression, (), {'filter':True, 'summary':False}),
    (pvc.CapData.rep_cond, (), {}),
    (pvc.CapData.filter_irr, (0.5, 1.5), {'ref_val':'self_val'}),
    (pvc.CapData.fit_regression, (), {}),
]

In [ ]:
pvc.run_test(das, measured_filters)

In [ ]:
das.get_summary()

In [ ]:
das_scatter = das.scatter_filters()
das_scatter

In [ ]:
das.timeseries_filters().opts(width=1200)

## Load and Filter PVsyst Data

To load and filter the modeled data, often from PVsyst, we simply create a new CapData object, load the PVsyst data, and apply the filtering methods as appropriate.

In [ ]:
sim = pvc.CapData('sim')

To load pvsyst data we use the `load_data` method with the `load_pvsyst` option set to True.  By default the `load_data` method will search for a csv file that includes `pvsyst` in the filename in a `data` directory in the same directory as this file.  If you have saved the pvsyst file in a different location, you can use the `path` and `fname` arguments to load it.

In [ ]:
sim.load_data(load_pvsyst=True)

In [ ]:
sim.column_groups

In [ ]:
sim.set_regression_cols(power='real_pwr--', poa='irr-poa-', t_amb='temp-amb-', w_vel='wind--')

In [ ]:
# Write over cptest.flt_sim dataframe with a copy of the original unfiltered dataframe
sim.reset_filter()

In [ ]:
simulated_filters = [
    (pvc.CapData.filter_time, (), {'test_date':'10/11/1990', 'days':60}),
    (pvc.CapData.filter_irr, (200, 930), {}),
    (pvc.CapData.filter_pvsyst, (), {}),
    (pvc.CapData.filter_irr, (0.5, 1.5), {'ref_val':das.rc['poa'][0]}),
    (pvc.CapData.fit_regression, (), {}),
]

In [ ]:
pvc.run_test(sim, simulated_filters)

In [ ]:
sim.get_summary()

In [ ]:
sim_scatter = sim.scatter_filters()
sim_scatter

## Results

The `get_summary` and `captest_results_check_pvalues` functions display the results of filtering on simulated and measured data and the final capacity test results comparing measured capacity to expected capacity, respectively.

In [ ]:
pvc.get_summary(das, sim)

In [ ]:
das.rc

In [ ]:
# the default hrs_req value is shown
das.print_points_summary(hrs_req=12.5)

In [ ]:
pvc.captest_results_check_pvalues(sim, das, 6000, '+/- 7', print_res=True)

The `overlay_scatters` function can be used to overlay the final scatter plots from scatter plots of all filtering steps produced above.

In [ ]:
pvc.overlay_scatters(das_scatter, sim_scatter)